In [1]:
import time
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from urllib.parse import urljoin
from selenium import webdriver
from bs4 import BeautifulSoup

def extract_reviews(html_content):
    if not html_content:
        print("No HTML content to process.")
        return []

    soup = BeautifulSoup(html_content, 'html.parser')
    
    # Extracting star rating
    star_rating_container = soup.find('section', class_='pr-review-snapshot-block-snippet')
    if star_rating_container:
        star_rating = star_rating_container.find('div', class_='pr-snippet-stars')
        star_rating_text = star_rating_container.find('div', class_='pr-snippet-rating-decimal').text.strip()
        if star_rating:
            star_rating_label = star_rating['aria-label']
            print(f"Star Rating: {star_rating_label}, {star_rating_text}")
        else:
            print("Star Rating not found.")
    else:
        print("Star Rating container not found.")

    # Extracting recommendation percentage
    recommendation_section = soup.find('section', class_='pr-review-snapshot-block-recommend')
    if recommendation_section:
        recommendation_percent = recommendation_section.find('span', class_='pr-reco-value').text.strip()
        print(f"Recommendation Percentage: {recommendation_percent}")
    else:
        print("Recommendation section not found.")

    # Extracting reviews
    reviews = soup.find_all('section', class_='pr-rd-content-block')
    reviews_data = []

    for idx, review in enumerate(reviews, start=1):
        print(f"\nProcessing review {idx}:")
        
        # Extracting review text
        review_text = review.find('p', class_='pr-rd-description-text')
        if review_text:
            print("Review Text:", review_text.text.strip())
        else:
            print("Review Text not found.")
        
        # Adding review data
        reviews_data.append({
            'Star Rating': star_rating_label if star_rating else None,
            'Rating Text': star_rating_text if star_rating_text else None,
            'Review Text': review_text.text.strip() if review_text else None
        })

    return reviews_data


# Selenium setup
options = Options()
options.add_argument("--headless")
options.add_argument("--disable-gpu")
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

def get_page_soup(url):
    print(f"Getting page soup for URL: {url}")
    driver.get(url)
    time.sleep(1)  # Wait for the page to load
    html = driver.page_source
    return BeautifulSoup(html, 'html.parser')

def search_product(product_id):
    print(f"Searching for product ID: {product_id}")
    search_url = f'https://www.zoro.com/search?q={product_id}'
    soup = get_page_soup(search_url)

    # Find the specific product link using a precise CSS selector
    for a in soup.select('a.product-card-image__link'):
        print(f"Checking link: {a['href']}")
        if product_id.lower() in a['href'].lower():
            product_url = urljoin(search_url, a['href'])
            print(f"Found product URL: {product_url}")
            return product_url

    print("Product URL not found.")
    return None

def find_reviews_link(product_url):
    print(f"Finding reviews link on product URL: {product_url}")
    soup = get_page_soup(product_url)

    # Find the reviews link using a precise CSS selector
    for a in soup.select('a'):
        print(f"Checking link: {a['href']}")
        if 'reviews' in a['href']:
            reviews_url = urljoin(product_url, a['href'])
            print(f"Found reviews URL: {reviews_url}")
            return reviews_url

    print("Reviews URL not found.")
    return None

def navigate_to_reviews(product_id):
    print(f"Navigating to reviews for product ID: {product_id}")
    product_url = search_product(product_id)
    if product_url:
        reviews_url = find_reviews_link(product_url)
        if reviews_url:
            print(f"Navigating to reviews URL: {reviews_url}")
            driver.get(reviews_url)
            time.sleep(1)  # Wait for the reviews page to load
            # Extract page source after page load
            html_content = driver.page_source
        
            # Pass HTML content to extract_reviews function
            reviews_data = extract_reviews(html_content)
            
            # Print extracted reviews data
            print("\nExtracted Reviews:")
            for review in reviews_data:
                print(review)
        else:
            print("Reviews URL not found.")
    else:
        print("Product URL not found.")

product_id = '1VCE8'
navigate_to_reviews(product_id)

driver.quit()


/bin/sh: /Applications/Google Chrome.app/Contents/MacOS/Google Chrome: No such file or directory
/bin/sh: /Applications/Google Chrome.app/Contents/MacOS/Google Chrome: No such file or directory


AttributeError: 'NoneType' object has no attribute 'split'